In [2]:
import numpy as np
import pandas as pd
import networkx as nx 
import random
import matplotlib.pyplot as plt

In [3]:
np.random.seed(1)
random.seed(1)

Functions that will be used:

In [ ]:
def one_iteration(G, gamma, beta) : 
    

We set the parameters of the model:

In [4]:
total_pop = 1000
p_0 = 0.01
gamma = 0.2
beta = 0.7

We create the graphs for each model

In [5]:
G_complete = nx.complete_graph(total_pop)
G_BA = nx.barabasi_albert_graph(total_pop, m = 5, seed = 1)
G_WS = nx.watts_strogatz_graph(total_pop, k = 4, p = 0.2, seed = 1)
# G_tree = nx.tree_graph(total_pop, seed = 1)
# G_star = nx.star_graph(total_pop, seed = 1)
# G_lattice = nx.latt
G_ER = nx.erdos_renyi_graph(total_pop, p = 0.5, seed = 1)




In [6]:
nx.set_node_attributes(G_complete, "Susceptible", "Status")
nx.set_node_attributes(G_BA, "Susceptible", "Status")
nx.set_node_attributes(G_WS, "Susceptible", "Status")
nx.set_node_attributes(G_ER, "Susceptible", "Status")

In [7]:
infected_0 = np.random.choice(total_pop, int(p_0*total_pop), replace=False)
infected_0

array([507, 818, 452, 368, 242, 929, 262, 810, 318,  49])

In [ ]:
# G_complete.nodes(data =True)
# G_BA.nodes(data =True)
# G_WS.nodes(data =True)
# G_ER.nodes(data =True)


We set the infected people at time 0

In [8]:
for i in infected_0 : 
    G_complete.nodes[i]["Status"] = "Infected"
    G_BA.nodes[i]["Status"] = "Infected"
    G_WS.nodes[i]["Status"] = "Infected"
    G_ER.nodes[i]["Status"] = "Infected"

In [9]:
infected_neightbour_i = len([v for v in nx.neighbors(G_complete, 0) if G_complete.nodes[v]["Status"] == "Infected"])
# infected_neightbour_i = len([v for v in nx.neighbors(G_BA, 0) if G_BA.nodes[v]["Status"] == "Infected"])
# infected_neightbour_i = len([v for v in nx.neighbors(G_WS, 0) if G_BA.nodes[v]["Status"] == "Infected"])
# infected_neightbour_i = len([v for v in nx.neighbors(G_ER, 0) if G_BA.nodes[v]["Status"] == "Infected"])

In [10]:
infected_neightbour_i

10

Code that computes an iteration of the epidemic:

In [30]:
list_healthy = []
list_infected = []
# Maybe there is abetter way to do that
for node, attribute in nx.get_node_attributes(G_complete, "Status").items() : 
    if attribute == "Infected" : 
        list_infected.append(node)
    else : list_healthy.append(node)

In [29]:
# Another way to do the same as above
# list_infected_2 = [u for (u,d) in nx.get_node_attributes(G_complete, "Status").items() if d == 'Infected']

In [ ]:
# CREATE A LIST WITH THE INFECTED NODES THAT WILL RECOVER IN THIS STEP
list_recovered = []
for i in list_infected : 
    if random.uniform(0,1) < gamma : 
        list_recovered.append(i)

# CREATE A LIST WOTH THE HEALTHY NODES THAT WILL BE INFECTED IN THIS STEP
for i in list_healthy : 
    neighbours_infected = [v for v in nx.neighbors(G_complete, 0) if G_complete.nodes[v]["Status"] == "Infected"]
    if not (random.uniform(0,1) < pow((1-beta),len(neighbours_infected))) :
        G_complete.nodes[i]["Status"] = "Infected" # (instead of creating a list we directly change their status)

for i in list_recovered : 
    G_complete.nodes[i]["Status"] = "Susceptible"